In [ ]:
!pip install langchain
!pip install pypdf
!pip install python-dotenv
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 977.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import getpass
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")

Enter your LangChain API key: ··········


In [ ]:
%pip install --upgrade --quiet  langchain-openai tiktoken chromadb langchain
import dotenv
dotenv.load_dotenv()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.4 MB/s eta 0:00:00
  

True

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "True"

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("le_petit_prince.pdf")
pages = loader.load()


pages = pages[0:106]

text = ""

for page in pages:
    text += page.page_content

text = text.replace('\t', ' ')

In [ ]:
num_tokens = llm.get_num_tokens(text)

print (f"This book has {num_tokens} tokens in it")

This book has 26053 tokens in it


In [ ]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=10000, chunk_overlap=3000)

docs = text_splitter.create_documents([text])

In [ ]:
num_documents = len(docs)

print (f"Now our book is split up into {num_documents} documents")

Now our book is split up into 13 documents


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectors = embeddings.embed_documents([x.page_content for x in docs])


In [ ]:
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=42).fit(vectors)

In [ ]:
kmeans.labels_

array([9, 2, 0, 8, 6, 4, 5, 1, 1, 7, 3, 3, 3], dtype=int32)

In [ ]:
closest_indices = []
for i in range(num_clusters):
    distances = np.linalg.norm(vectors - kmeans.cluster_centers_[i], axis=1)
    closest_index = np.argmin(distances)
    closest_indices.append(closest_index)

In [ ]:
selected_indices = sorted(closest_indices)
selected_indices

[0, 1, 2, 3, 4, 5, 6, 7, 9, 11]

In [ ]:
llm3 = ChatOpenAI(temperature=0,
                 openai_api_key=openai_api_key,
                 max_tokens=1000,
                 model='gpt-4o'
                )

In [ ]:
from langchain_core.prompts import PromptTemplate
map_prompt = """
You will be given a single passage of a book. This section will be enclosed in triple backticks (```)
Your goal is to give a summary in french of this section so that a reader will have a full understanding of what happened.
Your response should be at least three paragraphs and fully encompass what was said in the passage.

```{text}```
FULL SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [ ]:
map_chain = load_summarize_chain(llm=llm3,
                             chain_type="stuff",
                             prompt=map_prompt_template)

In [ ]:
selected_docs = [docs[doc] for doc in selected_indices]

In [ ]:
summary_list = []
for i, doc in enumerate(selected_docs):
    chunk_summary = map_chain.run([doc])
    summary_list.append(chunk_summary)

    print (f"Summary #{i} (chunk #{selected_indices[i]}) - Preview: {chunk_summary[:150]} \n")

Summary #0 (chunk #0) - Preview: Dans le premier chapitre du livre "Le Petit Prince" d'Antoine de Saint-Exupéry, l'auteur raconte une expérience de son enfance. À l'âge de six ans, il 

Summary #1 (chunk #1) - Preview: Dans ce passage du livre "Le Petit Prince" d'Antoine de Saint-Exupéry, le narrateur raconte sa rencontre avec un petit garçon extraordinaire dans le d 

Summary #2 (chunk #2) - Preview: Dans ce passage du "Petit Prince" d'Antoine de Saint-Exupéry, l'auteur critique la manière dont les adultes perçoivent le monde, en se concentrant sur 

Summary #3 (chunk #3) - Preview: Dans ce passage du "Petit Prince" de Saint-Exupéry, l'auteur explique pourquoi il n'y a pas d'autres dessins aussi impressionnants que celui des baoba 

Summary #4 (chunk #4) - Preview: Dans ce passage du "Petit Prince" de Saint-Exupéry, nous assistons à une série d'événements marquants dans la relation entre le petit prince et sa fle 

Summary #5 (chunk #5) - Preview: Dans ce passage du livre "Le Petit Pr

In [ ]:
summaries = "\n".join(summary_list)
summaries = Document(page_content=summaries)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

Your total summary has 4727 tokens


In [ ]:
summaries

Document(page_content='Dans le premier chapitre du livre "Le Petit Prince" d\'Antoine de Saint-Exupéry, l\'auteur raconte une expérience de son enfance. À l\'âge de six ans, il découvre une image fascinante dans un livre sur la Forêt Vierge, représentant un serpent boa avalant un fauve. Inspiré, il dessine son propre serpent boa digérant un éléphant et montre son dessin aux adultes, qui le prennent pour un chapeau. Déçu par leur manque de compréhension, il dessine l\'intérieur du serpent pour clarifier, mais les adultes lui conseillent de se concentrer sur des sujets plus "sérieux" comme la géographie et l\'histoire. Découragé, il abandonne sa carrière de peintre et devient pilote d\'avion. Il explique que, malgré ses nombreux voyages et rencontres avec des adultes, il n\'a jamais trouvé quelqu\'un qui comprenne vraiment ses dessins.\n\nDans le deuxième chapitre, l\'auteur raconte une panne de moteur dans le désert du Sahara, six ans avant l\'écriture du livre. Seul et sans aide, il te